In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import html
import scrapy
from time import sleep
import urllib3
import json
from selenium import webdriver
import random

In [3]:
def parser(link):

    encabezados = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
    }

    resp = requests.get(link, headers=encabezados, verify=False)
    resp = resp.text

    #soup = get_Soup('https://minciencias.gov.co/convocatorias/todas')
    parser = html.fromstring(resp)
    urllib3.disable_warnings()

    return parser

In [4]:
def paginas(link):
    parser_pag = parser(link)

    paginas = parser_pag.xpath('//a[@class="page-numbers"]//@href')
    paginas.append(link)

    return paginas

In [5]:
pag = 'https://www.senacyt.gob.pa/category/convocatorias/'

def links_proyectos(link):

    links_proyectos_final = []
    paginacion = paginas(link)

    for pag in paginacion:
        links = parser(pag)
        links_proyectos = links.xpath('//div[@class="post_content"]//a/@href')
        links_proyectos2 = links.xpath('//div[@class="post_no_thumb_content"]//a/@href')

        links_proyectos += links_proyectos2


        for url in links_proyectos:
            if 'author' not in url and 'beca' not in url:
                links_proyectos_final.append(url)

    return links_proyectos_final


def titulo_proyecto(link_proyecto):
    link_proyecto = link_proyecto
    parser_proyecto = parser(link_proyecto)

    titulo = parser_proyecto.xpath('//h1/text()')
    
    return titulo[0]


def descripcion_proyecto(link_proyecto):

    link_proyecto = link_proyecto

    parser_proyecto = parser(link_proyecto)
    descripcion = parser_proyecto.xpath('//div[@class="post_content"]//p//text()')
    descripcion_completa = ''

    for i in descripcion:
        descripcion_completa = descripcion_completa + i.strip() + ' '

    return descripcion_completa.strip()

def pdfs_proyecto(link_proyecto):

    link_proyecto = link_proyecto

    parser_proyecto = parser(link_proyecto)
    pdfs_pag = parser_proyecto.xpath('//div[@class="su-table su-table-responsive su-table-alternate"]//a/@href')

    pdfs = ''
    for i in pdfs_pag:
        if i.endswith('.pdf'):
            pdfs = pdfs + i.strip('"') + ', '

    return pdfs.strip(', ')

In [10]:
def panama():
    titulos = []
    descripciones = []
    titulos = []
    pdfs = []


    pag = 'https://www.senacyt.gob.pa/category/convocatorias/'
    links_proyectos_final = links_proyectos(pag)

    for link_proyecto in links_proyectos_final:
        
        try:
            titulo = titulo_proyecto(link_proyecto)
        except:
            totulo = ''
        
        try:
            descripcion = descripcion_proyecto(link_proyecto)
        except:
            descripcion = ''
            
        try:
            pdf = pdfs_proyecto(link_proyecto)
        except:
            pdf = ''

        titulos.append(titulo)
        descripciones.append(descripcion)
        pdfs.append(pdf)

        sleep(random.randrange(1, 3))
        
    panama = pd.DataFrame()
    panama['Titulos'] = titulos
    panama['Descripcion'] = descripciones
    panama['PDFs'] = pdfs
    panama['Link'] = links_proyectos_final

    return panama


In [11]:
panama = panama()
panama

KeyboardInterrupt: 